# Pair Programming 
## Lección 11 - Limpieza II - Strings
​
Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. **Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

variable    ¿Está limpia?

age         ❌ esta en formato string cuando debería ser integer y en algunos casos tenemos rangos de edad

species     ❌ es un jaleo! Debemos unificar los nombres y reducir a las especies más importantes

country     ✔️ los paises están en mayúsculas, algunos se repiten con algunos cambios

fatal       ✔️ la limpiamos en el pair de Pandas V

year        ✔️ es una columna de tipo float deberíamos convertirla a integer

sex         ✔️ la limpiamos en el pair de Pandas V

fecha       ✔️ la limpiamos en el pair de Pandas V

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

In [12]:
import pandas as pd
import numpy as np
import re

In [13]:
df = pd.read_csv("limpieza1.csv", index_col = 0)

In [14]:
df.head(2)

,year,type,country,activity,age,species_,mes,fatal,sex
0,2018,Boating,usa,Paddling,57,White shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M


1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

imaginemos que el valor de una celda es el siguiente:

'White shark, 3.5 m'

tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 

Un patrón que podriamos usar es: 
patron_blanco = r".*[Ww](hite|HITE).*"

esto podría ser así porque puede estar en mayúsculas o en minúsculas. 
​
de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [15]:
df["species_"].isnull().sum()

123

In [16]:
def especie_tiburon(string_especie):
    patron_blanco = "^white|White\s\w+"
    patron_tigre = "^tiger|Tiger\s\w+"
    patron_gris = "^grey|Grey\s\w+"
    patron_lemon = "^lemon|Lemon\s\w+"
    patron_toro = "^bull|Bull\s\w+"
    #patron_blanco = r".*[Ww](hite|HITE).*"
    #patron_tigre = r".*[Tt](iger|IGER).*"
    #patron_gris = r".*[Gg](rey|REY).*"
    #patron_lemon = r".*[Ll](emon|EMON).*"
    #patron_toro = r".*[Bb](ull|ULL).*"

    try:
        if re.findall(patron_blanco, string_especie):
            return "White shark"
        elif re.findall(patron_tigre, string_especie):
            return "Tiger shark"
        elif re.findall(patron_gris, string_especie):
            return "Grey shark"
        elif re.findall(patron_lemon, string_especie):
            return "Lemon shark"
        elif re.findall(patron_toro, string_especie):
            return "Bull shark"
        else:
            return "Unspecified"
    except:
        return np.nan

In [17]:
df["especie_tiburon"] = (df["species_"].apply(especie_tiburon))

In [18]:
df.sample(5)

,year,type,country,activity,age,species_,mes,fatal,sex,especie_tiburon
927,1993,Unprovoked,usa,Paddling on surfboard,35,"Tiger shark, 3.7 m [12'], (tooth fragment reco...",Mar,N,M,Tiger shark
1512,1780,Unprovoked,usa,Swimming,NaN,NaN,Aug,Y,M,NaN
1239,1965,Unprovoked,south africa,Paddling rescue ski,22,"White shark, 3.7 m [12']",Jan,N,M,White shark
818,1998,Unprovoked,bahamas,Spearfishing,28,"Caribbean reef shark, 1.2 m to 1.5 m [4' to 5']",Aug,N,M,Unspecified
437,2008,Unprovoked,australia,Body boarding,16,Bull shark,Apr,Y,M,Bull shark


Comprobamos que ahora en la columna especie_tiburon aparecen solo los nombre de los tiburones con el método unique.

In [31]:
df["especie_tiburon"].unique()

array(['White shark', 'Tiger shark', 'Bull shark', 'Grey shark',
       'Unspecified', 'Lemon shark', nan], dtype=object)

2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
- Puede que os salga un error similar a este:

TypeError: expected string or bytes-like object

Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

df['nombre_columna'] = df['nombre_columna'].astype(str)

- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, .
- Por último cambiad el tipo de la columna de string a integer.

In [20]:
df["age"].dtype

dtype('O')

In [21]:
df["age"] = df["age"].astype(str)

In [22]:
df["age"].dtype

dtype('O')

In [23]:
def edades(edad):
    patron_age = "\d+"
    try:
        return re.findall(patron_age, str(edad))[0]
    except:
        np.nan

In [24]:
df["age_ok"] = (df["age"].apply(edades))

In [25]:
df.head()

,year,type,country,activity,age,species_,mes,fatal,sex,especie_tiburon,age_ok
0,2018,Boating,usa,Paddling,57,White shark,Jun,N,F,White shark,57
1,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M,Tiger shark,18
2,2018,Unprovoked,usa,Walking,15,"Bull shark, 6'",May,N,M,Bull shark,15
3,2018,Provoked,australia,Feeding sharks,32,Grey reef shark,May,N,M,Grey shark,32
4,2018,Invalid,england,Fishing,21,Invalid incident,May,N,M,Unspecified,21


Comprobamos con un ejemplo de una fila con la columna "age" con un rango 13 or 14, se ha quedado en la columna "age_ok" con el primer número.

In [26]:
df[df["age"] == '13 or 14']

,year,type,country,activity,age,species_,mes,fatal,sex,especie_tiburon,age_ok
1635,0,Unprovoked,iraq,Swimming,13 or 14,Bull shark,NaN,Y,M,Bull shark,13


La columna age en algunos casos nos especifica un rango de edades, por lo que en la función edades, en el return le ponemos [0] es para que nos devuelva la posición 0 de nuestra lista.

In [27]:
df["age_ok"].dtype

dtype('O')

In [28]:
df["age_ok"] = df["age_ok"].astype('Int64', errors = 'raise')

In [29]:
df["age_ok"].dtype

Int64Dtype()

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.
Happy coding 🦈

In [30]:
df.to_csv("limpieza2.csv")